## Média Salários por escola

O Dataframe que será criado aqui no final né uma junção das informações do SIOPE 2017 e do Censo Escolar.

Preciso juntar esses dois dataframes para conseguir cruzar com as informações do IDEB, o que será feito em outro notebook.


### Dados do Censo Escolar:

In [2]:
import pandas as pd
import os
import janitor
import numpy as np

Criando o objeto escolas, que terá informações do Censo Escolar.
Eu vou juntar com os gastos do SIOPE de maneira a ter o Código da Escola

In [3]:
os.chdir('C:/Users/coliv/Documents/R-Projects/qualidade_gastos_educação/arquivos grandes (fora do git)/Microdados_Censo_Escolar_2017/DADOS/ESCOLAS')
escolas = pd.read_csv('escolas_sp.csv')

# Normalizando os dados:

#mudando o nchar de CO_MUNICIPIO para 6
escolas['CO_MUNICIPIO'] = escolas['CO_MUNICIPIO'].astype('str')
escolas['CO_MUNICIPIO'] = escolas['CO_MUNICIPIO'].str[0:6]

#Transofrmando variáveis em 'str'

for var in ['CO_UF', 'CO_MUNICIPIO', 'CO_ENTIDADE', 'NO_ENTIDADE']:
    escolas[var] = escolas[var].astype('str')

Agora abrindo o arquivo do SIOPE 

In [4]:
os.chdir('C:/Users/coliv/Documents/R-Projects/qualidade_gastos_educação/arquivos grandes (fora do git)/remuneracao_2017')
siope_2017_sp_original = pd.read_csv("siope_rem_prof_2017.csv", encoding = "ISO-8859-1")

Criando uma variável boolean que vai me dizer parte do salário pago ao funcionário registrado naquela linha é custeada com o fundeb (participação do fundeb verdadeira ou falsa)

In [7]:
siope_2017_sp_original['PARTICIPACAO_FUNDEB'] = np.where((siope_2017_sp_original.VL_PARC_MINIMA_FUNDEB > 0) |
                                                         (siope_2017_sp_original.VL_PARC_MAXIMA_FUNDEB > 0), 1, 0)

Agora vou criar mesmo o meu objeto com a média dos salários por escola e percentual de participação do fundeb no pagamento de salários:

In [8]:
siope_media = siope_2017_sp_original.groupby(['CO_UF', 
                                              'CO_MUNICIPIO', 
                                              'NO_MUNICIPIO', 
                                              'LOCAL_EXERCICIO'], as_index=False).agg({'VL_TOTAL' : 'mean',
                                                                                       'PARTICIPACAO_FUNDEB' : ['sum',
                                                                                                                'count']}).round()

siope_media.columns = ['CO_UF', 'CO_MUNICIPIO' , 'NO_MUNICIPIO',
                      'NO_ENTIDADE', 'SALARIO_MEDIO', 'QTDE_FUNDEB', 'QTDE_TOTAL']

siope_media['PERC_PART_FUNDEB'] = siope_media.QTDE_FUNDEB / siope_media.QTDE_TOTAL

siope_media = siope_media.drop(['QTDE_FUNDEB', 'QTDE_TOTAL'], axis=1)

for col in ['CO_UF', 'CO_MUNICIPIO', 'NO_ENTIDADE']:
    siope_media[col] = siope_media[col].astype('str')
    
# Retirando colunas dentro da lista proc
# Retirando as informações de pagamento que são para secretaria municipal ou prefeitura municipal:
proc = ['SEC MUN DE EDUC', 'PREFEITURA MUN']
siope_media = siope_media[~siope_media.NO_ENTIDADE.str.contains('|'.join(proc))]

Agora tenho que fazer o join.
Lembrando, a pergunta que eu quero fazer é quanto os municípios gastam por escola? Isso é comparável com o ideb?

Antes de fazer o join eu preciso alterar algumas denominações presentes na coluna 'NO_ENTIDADE' para garantir que o resultado do meu merge vai ser alto:

 - No SIOPE_media, algumas escolas tem o 'tipo' delas no final (EMEI< CMEI, ETC) e outras no começo. Vou apagar tudo

Na parte abaixo, o campo ```a = r'\b' + i + r'\b'``` é pra criar a REGEX que estabelece que o meu for loop vai procurar exatamente aquela expressão; depois o resultado 'a' desta regex é substituido nos dois loops mais abaixo.   

In [9]:
for i in ['ESCOLA DE EDUCACAO ESPECIAL APAE', 'ESCOLA DE EDUCACAO ESPECIAL', 'PROFESSORA', 'PROFESSOR','PROF.', 
          'PROF', 'CETEC', 'CEMEI', 'EMEIEF', 'ETEC', 'EMEB', 'EMEI', 'CMEI', 'EMEF', 'CEM', 'CEI', 
          'ESCOLA MUNICIPAL DE ED INFANTIL', 'CR P CONV',
          'CENTRO DE EDUCACAO INFANTIL', 'CENTRO MUN DE EDUCACAO INFANTIL', 'CRECHE', 'MUNICIPAL', 'MUN']:
    a = r'\b' + i + r'\b'   
    siope_media['NO_ENTIDADE'] = siope_media.NO_ENTIDADE.str.replace(a, '', regex=True)
    escolas['NO_ENTIDADE'] = escolas.NO_ENTIDADE.str.replace(a, '', regex=True)
    
# Substituindo dr:
for df in [siope_media, escolas]: 
    df['NO_ENTIDADE'] = df.NO_ENTIDADE.str.replace('DOUTORA', 'DRA', regex=False)
    df['NO_ENTIDADE'] = df.NO_ENTIDADE.str.replace('DOUTOR', 'DR', regex=False)
    
#Tirando os whitespaces:
for df in [siope_media, escolas]: 
    df['NO_ENTIDADE'] = df.NO_ENTIDADE.str.replace('(^\s+|\s+$)', '', regex=True)
    df['NO_ENTIDADE'] = df.NO_ENTIDADE.str.replace('\s+', ' ', regex=True)

Agora vamos ver se as colunas são realmente iguais.

In [10]:
print(escolas.dtypes, '\n \n \n', siope_media.dtypes)

Unnamed: 0                int64
CO_UF                    object
CO_MUNICIPIO             object
CO_ENTIDADE              object
TP_ETAPA_ENSINO         float64
MATRICULAS_POR_ETAPA      int64
NO_ENTIDADE              object
TEM_IDEB                   bool
dtype: object 
 
 
 CO_UF                object
CO_MUNICIPIO         object
NO_MUNICIPIO         object
NO_ENTIDADE          object
SALARIO_MEDIO       float64
PERC_PART_FUNDEB    float64
dtype: object


In [11]:
escolas_siope = escolas.merge(siope_media, on=['CO_UF','CO_MUNICIPIO','NO_ENTIDADE'], how='inner')

In [12]:
escolas_siope.shape

(43575, 11)

In [14]:
escolas_siope = escolas_siope.drop_duplicates()

In [15]:
escolas_siope.shape #Não tinha duplicata, não acredito!

(43575, 11)

O df do SIOPE, contendo os pagamentos médios por escolas, têm 12.082 escolas únicas registradas. Com essa limpeza de banco conseguimos realizar um match de 93,2%

Então meu DF final é uma tabela que contém as seguintes informações:
    
|coluna|info|
|--|--|
|CO_UF| Identificador de UF|
|CO_MUNICIPIO| Identificador de município com seis dígitos|
|TP_ETAPA_ENSINO|Etapa do ensino que constam as informações|
|Matrículas por etapa|De acordo com o censo escolar de 2017, quantas crianças foram matriculadas naquela etapa. Se < 20 não existe informação para o ideb nesta série|
|NO_ENTIDADE| Nome da escola|
|TEM_IDEB| Se de acordo com o número de crianças matriculadas vai ter info da prova do IDEB|
|NO_MUNICIPIO| Nome do Município|
|SALARIO_MEDIO| Média do salário pago aos profissionais de educação de acordo com as informações do SIOPE 2017|
|PERC_PART_FUNDEB| Percentual dos salários da escola que recebem recursos do FUNDEB|

In [16]:
escolas_siope.head(2)

,Unnamed: 0,CO_UF,CO_MUNICIPIO,CO_ENTIDADE,TP_ETAPA_ENSINO,MATRICULAS_POR_ETAPA,NO_ENTIDADE,TEM_IDEB,NO_MUNICIPIO,SALARIO_MEDIO,PERC_PART_FUNDEB
0,36,35,350010,35063113,1.0,56,SONHO DE CRIANCA CICLO I,True,Adamantina,3434.0,1.0
1,37,35,350010,35065262,2.0,172,EULALIA PASCHOAL BRIGHENTI CICLO II,True,Adamantina,5226.0,1.0


Agora vou exportar essa tabela antes que eu perca tudo (como aconteceu semana passada)

In [17]:
os.chdir('C:/Users/coliv/Documents/R-Projects/qualidade_gastos_educação/analise_gastos_siope/data/Cruzamentos')
escolas_siope.to_csv(r'media_salarios_por_escola.csv')